In [ ]:
TRAIN_DATA = "DATA/train.csv"
TEST_DATA = "DATA/test.csv"
SUBMISSON_DATA = "DATA/submission.csv"

In [ ]:
import pandas as pd
import statsmodels.api as sm

In [ ]:
df = pd.read_csv(TRAIN_DATA)
df

In [ ]:
tdf = df[df['Статус'] == 'Отменен']
tdf['Тип переклассификации'].value_counts()

In [ ]:
priority = {'3-Низкий': 3, '2-Средний': 2, '1-Высокий': 1, '0-Критический': 0}
status = {'Отменен': 0, 'Закрыт': 1}
obr = {'Запрос': 0, 'Инцидент': 1}
critical = {'4-Нет': 4, "3-Базовая": 3, "2-Повышенная": 2, "1-Особая": 1}
influence = {"4-Нет влияния": 4, "3-Малое": 3, "2-Значительное": 2, "1-Всеохватывающее": 1}

In [ ]:
df['Приоритет'] = df['Приоритет'].map(priority)
df

In [ ]:
df['Статус'] = df['Статус'].map(status)
df

In [ ]:
df['Тип обращения на момент подачи'] = df['Тип обращения на момент подачи'].map(obr)
df['Тип обращения итоговый'] = df['Тип обращения итоговый'].map(obr)
df

In [ ]:
df['Критичность'] = df['Критичность'].map(critical)
df['Влияние'] = df['Влияние'].map(influence)
df

In [ ]:
# df[df['Дата восстановления'].isna()] drop
df = df.dropna(subset=['Дата восстановления'])
df

In [ ]:
df = df.drop_duplicates(subset=['Дата закрытия обращения', 'Система', 'Место', 'Сервис'])

In [ ]:
df[df['Статус'] != 0]['Тип переклассификации'].value_counts()

In [ ]:
df['Тип переклассификации'].value_counts()

In [ ]:
target = df['Тип обращения итоговый']
features = df[['Приоритет', 'Тип обращения на момент подачи', 'Критичность', 'Статус', 'Влияние']]

In [ ]:
features = sm.add_constant(features)
model = sm.OLS(target, features)
result = model.fit()

In [ ]:
model2 = sm.Logit(target, features)
result2 = model2.fit()
print(result2.summary())

In [ ]:
print(result.summary())

In [ ]:
df = df[df['Статус'] != 0]

In [79]:
df.dtypes

Содержание                        object
Сервис                            object
Приоритет                          int64
Статус                             int64
Функциональная группа             object
Крайний срок                      object
Дата обращения                    object
Тип обращения на момент подачи     int64
Тип обращения итоговый             int64
Решение                           object
Тип переклассификации              int64
Дата восстановления               object
Дата закрытия обращения           object
Критичность                        int64
Влияние                            int64
Система                           object
Место                             object
dtype: object

In [80]:
df['Дата обращения'] = pd.to_datetime(df['Дата обращения'])
df['Дата закрытия обращения'] = pd.to_datetime(df['Дата закрытия обращения'])
df['range_time'] = df['Дата закрытия обращения'] - df['Дата обращения']
df

,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,rage_time
2,Выполнена разблокировка у/з REGION\*****,Сервис7,2,1,ФГ8,2018-01-31 07:51,2018-01-25 22:47:00,0,0,Выполнена разблокировка у/з Ne_bosS\*********\...,0,2018-01-25 22:47,2018-02-02 07:52:00,1,3,Система7,Место190,7 days 09:05:00
3,.Не включается ПК,АРМ,3,1,ФГ390,2018-01-28 05:50,2018-01-25 23:13:00,1,1,Выход из стоя БП. Заменён.,0,2018-01-25 23:55,2018-01-26 00:00:00,4,3,База,Место177,0 days 00:47:00
4,"Заявка № ***********, *******************",АРМ,3,1,ФГ303,2018-02-03 02:51,2018-01-25 23:39:00,0,0,Работы по обращению выполнены,0,2018-01-28 02:54,2018-02-04 07:53:00,4,4,База,Место216,9 days 08:14:00
5,"Заявка № ***********, *******************",АРМ,3,1,ФГ372,2018-01-31 07:51,2018-01-25 23:39:00,0,0,Доступ заблокирован,0,2018-01-26 03:19,2018-02-02 07:52:00,4,4,Система62,Место216,7 days 08:13:00
6,"Заявка № ***********, *******************",АРМ,3,1,ФГ303,2018-02-01 02:51,2018-01-25 23:39:00,0,0,Работы по обращению выполнены,0,2018-01-26 03:11,2018-02-02 07:52:00,4,4,База,Место216,7 days 08:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14797,Заявка на предоставление и отзыв прав доступа ...,Сервис11,3,1,ФГ13,2018-02-21 11:14,2018-02-16 12:59:00,0,0,Работы по обращению выполнены,0,2018-02-17 04:56,2018-02-24 07:54:00,4,4,Система11,Место163,7 days 18:55:00
14799,Устранение уязвимостей ИБ в соответствии с Пла...,АРМ,3,1,ФГ135,2018-02-21 14:36,2018-02-16 13:07:00,0,0,Выполнено,0,2018-02-21 12:03,2018-03-07 13:22:00,4,3,База,Место3,19 days 00:15:00
14800,Контроль ПУ. Заявка на доступ к портам.,Сервис2,3,1,ФГ315,2018-02-21 14:23,2018-02-16 13:08:00,0,0,Работы выполнены согласно заявке.,0,2018-02-21 07:01,2018-03-07 08:46:00,4,4,Система2,Место163,18 days 19:38:00
14801,Ремонт стульев.,Сервис73,3,1,ФГ159,2018-02-21 14:24,2018-02-16 13:09:00,0,0,выполнено,0,2018-02-17 10:01,2018-02-24 11:22:00,4,4,Система71,Место3,7 days 22:13:00


In [82]:
df['range_time'] = df['range_time'].dt.total_seconds() / 3600
df

,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,rage_time
2,Выполнена разблокировка у/з REGION\*****,Сервис7,2,1,ФГ8,2018-01-31 07:51,2018-01-25 22:47:00,0,0,Выполнена разблокировка у/з Ne_bosS\*********\...,0,2018-01-25 22:47,2018-02-02 07:52:00,1,3,Система7,Место190,177.083333
3,.Не включается ПК,АРМ,3,1,ФГ390,2018-01-28 05:50,2018-01-25 23:13:00,1,1,Выход из стоя БП. Заменён.,0,2018-01-25 23:55,2018-01-26 00:00:00,4,3,База,Место177,0.783333
4,"Заявка № ***********, *******************",АРМ,3,1,ФГ303,2018-02-03 02:51,2018-01-25 23:39:00,0,0,Работы по обращению выполнены,0,2018-01-28 02:54,2018-02-04 07:53:00,4,4,База,Место216,224.233333
5,"Заявка № ***********, *******************",АРМ,3,1,ФГ372,2018-01-31 07:51,2018-01-25 23:39:00,0,0,Доступ заблокирован,0,2018-01-26 03:19,2018-02-02 07:52:00,4,4,Система62,Место216,176.216667
6,"Заявка № ***********, *******************",АРМ,3,1,ФГ303,2018-02-01 02:51,2018-01-25 23:39:00,0,0,Работы по обращению выполнены,0,2018-01-26 03:11,2018-02-02 07:52:00,4,4,База,Место216,176.216667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14797,Заявка на предоставление и отзыв прав доступа ...,Сервис11,3,1,ФГ13,2018-02-21 11:14,2018-02-16 12:59:00,0,0,Работы по обращению выполнены,0,2018-02-17 04:56,2018-02-24 07:54:00,4,4,Система11,Место163,186.916667
14799,Устранение уязвимостей ИБ в соответствии с Пла...,АРМ,3,1,ФГ135,2018-02-21 14:36,2018-02-16 13:07:00,0,0,Выполнено,0,2018-02-21 12:03,2018-03-07 13:22:00,4,3,База,Место3,456.250000
14800,Контроль ПУ. Заявка на доступ к портам.,Сервис2,3,1,ФГ315,2018-02-21 14:23,2018-02-16 13:08:00,0,0,Работы выполнены согласно заявке.,0,2018-02-21 07:01,2018-03-07 08:46:00,4,4,Система2,Место163,451.633333
14801,Ремонт стульев.,Сервис73,3,1,ФГ159,2018-02-21 14:24,2018-02-16 13:09:00,0,0,выполнено,0,2018-02-17 10:01,2018-02-24 11:22:00,4,4,Система71,Место3,190.216667


In [87]:
system_l = df['Система'].unique()
system_d = {'База': 0}
for i in system_l:
    if i not in system_d.keys():
        system_d[i] = int(i[7:])
system_d

{'База': 0,
 'Система7': 7,
 'Система62': 62,
 'Система21': 21,
 'Система58': 58,
 'Система304': 304,
 'Система13': 13,
 'Система94': 94,
 'Система218': 218,
 'Система133': 133,
 'Система138': 138,
 'Система2': 2,
 'Система97': 97,
 'Система43': 43,
 'Система18': 18,
 'Система101': 101,
 'Система23': 23,
 'Система22': 22,
 'Система25': 25,
 'Система11': 11,
 'Система125': 125,
 'Система64': 64,
 'Система164': 164,
 'Система26': 26,
 'Система71': 71,
 'Система35': 35,
 'Система33': 33,
 'Система38': 38,
 'Система44': 44,
 'Система153': 153,
 'Система41': 41,
 'Система134': 134,
 'Система73': 73,
 'Система61': 61,
 'Система76': 76,
 'Система100': 100,
 'Система368': 368,
 'Система92': 92,
 'Система8': 8,
 'Система52': 52,
 'Система24': 24,
 'Система70': 70,
 'Система341': 341,
 'Система120': 120,
 'Система20': 20,
 'Система36': 36,
 'Система145': 145,
 'Система130': 130,
 'Система141': 141,
 'Система65': 65,
 'Система48': 48,
 'Система17': 17,
 'Система12': 12,
 'Система135': 135,
 'Сист

In [88]:
df['Система'] = df['Система'].map(system_d)
df

,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Решение,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,range_time
2,Выполнена разблокировка у/з REGION\*****,Сервис7,2,1,ФГ8,2018-01-31 07:51,2018-01-25 22:47:00,0,0,Выполнена разблокировка у/з Ne_bosS\*********\...,0,2018-01-25 22:47,2018-02-02 07:52:00,1,3,7,Место190,177.083333
3,.Не включается ПК,АРМ,3,1,ФГ390,2018-01-28 05:50,2018-01-25 23:13:00,1,1,Выход из стоя БП. Заменён.,0,2018-01-25 23:55,2018-01-26 00:00:00,4,3,0,Место177,0.783333
4,"Заявка № ***********, *******************",АРМ,3,1,ФГ303,2018-02-03 02:51,2018-01-25 23:39:00,0,0,Работы по обращению выполнены,0,2018-01-28 02:54,2018-02-04 07:53:00,4,4,0,Место216,224.233333
5,"Заявка № ***********, *******************",АРМ,3,1,ФГ372,2018-01-31 07:51,2018-01-25 23:39:00,0,0,Доступ заблокирован,0,2018-01-26 03:19,2018-02-02 07:52:00,4,4,62,Место216,176.216667
6,"Заявка № ***********, *******************",АРМ,3,1,ФГ303,2018-02-01 02:51,2018-01-25 23:39:00,0,0,Работы по обращению выполнены,0,2018-01-26 03:11,2018-02-02 07:52:00,4,4,0,Место216,176.216667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14797,Заявка на предоставление и отзыв прав доступа ...,Сервис11,3,1,ФГ13,2018-02-21 11:14,2018-02-16 12:59:00,0,0,Работы по обращению выполнены,0,2018-02-17 04:56,2018-02-24 07:54:00,4,4,11,Место163,186.916667
14799,Устранение уязвимостей ИБ в соответствии с Пла...,АРМ,3,1,ФГ135,2018-02-21 14:36,2018-02-16 13:07:00,0,0,Выполнено,0,2018-02-21 12:03,2018-03-07 13:22:00,4,3,0,Место3,456.250000
14800,Контроль ПУ. Заявка на доступ к портам.,Сервис2,3,1,ФГ315,2018-02-21 14:23,2018-02-16 13:08:00,0,0,Работы выполнены согласно заявке.,0,2018-02-21 07:01,2018-03-07 08:46:00,4,4,2,Место163,451.633333
14801,Ремонт стульев.,Сервис73,3,1,ФГ159,2018-02-21 14:24,2018-02-16 13:09:00,0,0,выполнено,0,2018-02-17 10:01,2018-02-24 11:22:00,4,4,71,Место3,190.216667


In [92]:
func_l = df['Функциональная группа'].unique()
func_d = {j: int(j[2:]) for j in func_l}
func_d

ValueError: invalid literal for int() with base 10: 'Система268'

In [89]:
df.to_csv("DATA/new_train.csv", index=False)